# 양방향 LSTM

- RNN이나 LSTM은 시퀀스 또는 시계열 데이터 처리에 특화되어 은닉층에서 과거의 정보를 기억할 수 있음
- 그러나 순환 구조의 특성상 데이터가 입력 순으로 처리되기 때문에 이전 시점의 정보만 활용할 수 밖에 없다는 단점이 존재
    - 문장이 길어질수록 성능이 저하
    - 예) ios앱 (개발)은 맥북이 필요합니다.
    - 위의 경우에 ios와 앱 이라는 단어만으로는 개발 이라는 단어를 유추하기 힘듦
        - 문장의 앞부분보다 뒷부분에 중요한 정보가 있음
        
- 양방향 LSTM(Bidirectional LSTM)
    - 기존 LSTM 계층에 역방향으로 처리하는 LSTM 계층을 추가해 양방향에서 문장의 패턴을 분서석할 수 있도록 구성
    - 입력 문장을 양방향에서 처리하기 때문에 시퀀스 길이가 길어져도 정보 손실 없이 처리가 가능
    
<img src = "./image/Bidirectional_LSTM.jpg">

# 개체명 인식(Named Entity Recognition)

- 각 개체의 유형을 인식
- 문장 내에 포함된 어떤 단어가 인물, 장소, 날짜 등을 의미하는 단어인지 인식하는 것
- 개체명 인식은 챗봇에서 문장을 정확하게 해석하기 위해 반드시 처리해야 하는 전처리 과정임
    - 예) 날짜와 지역에 대해 개체 인식을 할 수 있는 모델이 있다고 가정할 경우
    - 입력 문장 : 내일 부산 날씨 알려줘
    - 문장 의도 : 날씨 요청
    - 개체명 인식 결과 : 내일 = 날짜, 부산 = 지역
    
- 단순한 질문 형태라면 개체명 사전을 구축해 해당 단어들과 매핑되는 개체명을 찾을 수도 있음
    - 문장 구조가 복잡하거나 문맥에 따라 단어의 의미가 바뀐다면 딥러닝 모델을 활용해야 함
    
- 개체명 사전 구축 방식은 신조어나 사전에 포함되지 않은 단어는 처리 불가능하며 사람이 직접 사전 데이터를 관리해야 하기 때문에 관리비용이 많이 필요함

## BIO 표기법

- Beginning, Inside, Outside 의 약자
- 각 토큰마다 태그를 붙이기 위해 사용
- Beginning : 개체명이 시작되는 단어에 "B-개체명"으로 태그
- Inside : "B-개체명"과 연결되는 단어일 때 "I-개체명"으로 태그
- Outside : 개체명 이외의 모든 토큰에 태그
    - 예) 오늘부터 길동 포터는 삼성전자에 근무합니다.
    - 오늘 -> B-date
    - 부터 -> O
    - 길동 -> B-Person
    - 포터 -> I-Person
    - 는 -> O
    - 삼성 -> B-Company
    - 전자 -> I-Company
    - 에 -> O
    - 근무 -> O
    - 합니다. -> O
    
- 2 개 이상의 토큰이 하나의 개체를 구성하는 경우가 많기 때문에 BIO 표기법을 사용

## 국립국어원 언어정보나눔터 개체명 인식 모델을 위한 말뭉치

- ; 으로 시작하는 문장 : 원본 문장
- $ 로 시작하는 문장 : 해당 문장에서 NER 처리된 결과
- 개체명 인식 모델은 단어 토큰을 입력했을 때 출력되는 NER 태그값을 예측
    - 예) "삼성전자"를 입력한다면 B_OG(단체) 태그가 출력되도록 학습

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional)
from tensorflow.keras.optimizers import Adam
from seqeval.metrics import f1_score, classification_report

In [4]:
# 학습 파일 불러오기
def read_file(file_name):
    sent = []
    with open(file_name, "r", encoding = "utf-8") as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ";" and lines[idx + 1][0] == "$":
                this_sent = []
                
            elif l[0] == "$" and lines[idx - 1][0] == ";":
                continue
                
            elif l[0] == "\n":
                sent.append(this_sent)
                
            else:
                this_sent.append(tuple(l.split()))
                
    return sent

In [5]:
# 학습용 말뭉치 데이터를 불러옴
corpus = read_file("./data/ner/train.txt")

In [6]:
corpus[0]

[('1', '한편', 'NNG', 'O'),
 ('1', ',', 'SP', 'O'),
 ('2', 'AFC', 'SL', 'O'),
 ('2', '챔피언스', 'NNG', 'O'),
 ('2', '리그', 'NNG', 'O'),
 ('3', 'E', 'SL', 'B_OG'),
 ('3', '조', 'NNG', 'I'),
 ('3', '에', 'JKB', 'O'),
 ('4', '속하', 'VV', 'O'),
 ('4', 'ㄴ', 'ETM', 'O'),
 ('5', '포항', 'NNP', 'O'),
 ('6', '역시', 'MAJ', 'O'),
 ('7', '대회', 'NNG', 'O'),
 ('8', '8강', 'NNG', 'O'),
 ('9', '진출', 'NNG', 'O'),
 ('9', '이', 'JKS', 'O'),
 ('10', '불투명', 'NNG', 'O'),
 ('10', '하', 'VV', 'O'),
 ('10', '다', 'EC', 'O'),
 ('11', '.', 'SF', 'O')]

In [7]:
len(corpus)

3555

In [9]:
# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentences = []
    sentence, bio_tag = [], []
    for w in t:
        sentence.append(w[1])
        bio_tag.append(w[3])
        
    sentences.append(sentence)
    tags.append(bio_tag)

- 단어와 BIO 태그만 이용해 학습용 데이터셋을 생성

In [10]:
print("샘플 크기 : ", len((sentences)))
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 :", (sum(map(len, sentences)) / len(sentences)))

샘플 크기 :  3555
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302


In [11]:
# 토크나이저 정의

# 단어 사전의 첫 번째 인덱스에는 OOV사용
sent_tokenizer = preprocessing.text.Tokenizer(oov_token = "OOV")
sent_tokenizer.fit_on_texts(sentences)

tag_tokenizer = preprocessing.text.Tokenizer(lower = False) # 태그 정보는 소문자로 변환하지 않음
tag_tokenizer.fit_on_texts(tags)

In [12]:
# 단어 사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 8
단어 사전 크기 : 13834


In [13]:
tag_tokenizer.word_index

{'O': 1, 'I': 2, 'B_OG': 3, 'B_PS': 4, 'B_DT': 5, 'B_LC': 6, 'B_TI': 7}

In [14]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)

In [15]:
print(x_train[0])
print(y_train[0])

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = "PAD"

In [17]:
index_to_ner

{1: 'O',
 2: 'I',
 3: 'B_OG',
 4: 'B_PS',
 5: 'B_DT',
 6: 'B_LC',
 7: 'B_TI',
 0: 'PAD'}

In [19]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding = "post", maxlen = max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding = "post", maxlen = max_len)

In [20]:
# 학습데이터와 테스트데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 12)

In [21]:
# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes = tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = tag_size)

In [24]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 8)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40, 8)


In [23]:
x_train[0]

array([ 479,    6,   33,  327, 2615,    9, 2207,    4,   18,   62, 6379,
          5, 6380,    2,    6,  183, 4295,   10,   57,    7, 6381,  186,
        611,   10,   34, 1368,  500,   29,   38,    4,    8,    3,    0,
          0,    0,    0,    0,    0,    0,    0])

In [26]:
# 모델 정의(Bi-LSTM)
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 30, input_shape = (max_len,), mask_zero = True))
model.add(Bidirectional(LSTM(200, return_sequences = True, dropout = 0.5, recurrent_dropout = 0.25)))
model.add(TimeDistributed(Dense(tag_size, activation = "softmax")))

- mask_zero = True : 0 으로 패딩된 값을 마스킹하여 네트워크의 뒤로 전달되지 않게 만듦
- TimeDistributed : many-to-many 로 동작. 각 타임스텝마다 출력이 있어야 함

<img src = "./image/time_distributed.jpg">

In [27]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 40, 30)              │         415,020 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 40, 400)             │         369,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 40, 8)               │           3,208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 787,828 (3.01 MB)

 Trainable params: 787,828 (3.01 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(loss = "categorical_crossentropy", optimizer = Adam(0.01), metrics = ["accuracy"])

In [29]:
model.fit(x_train, y_train, batch_size = 128, epochs = 10)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 180ms/step - accuracy: 0.5806 - loss: 1.0008
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.6339 - loss: 0.3561
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.6616 - loss: 0.2314
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 171ms/step - accuracy: 0.6684 - loss: 0.1857
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 0.6805 - loss: 0.1503
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 0.6868 - loss: 0.1204
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 0.6859 - loss: 0.0931
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.6919 - loss: 0.0744
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.7052 - loss: 0.0662
Epoch 10/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.6966 - loss: 0.0579


In [30]:
model.evaluate(x_test, y_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6761 - loss: 0.2341


[0.242510125041008, 0.6661040782928467]

- BIO 태그는 실제 의미 있는 태그보다 의미 없는 O 태그가 대부분을 차지하고 있기 때문에 실제 성능과 무관하게 높은 점수가 나올 수 있음
- 따라서 개체명 인식에서는 F1스코어가 주로 이용됨
- F1 스코어 : 정밀도와 재현율의 조화 평균

In [36]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
        
    return result

In [38]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test) # (711, 40) -> model -> (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted)
test_tags = sequences_to_tag(y_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [39]:
y_predicted[0]

array([[1.46292622e-08, 9.99930501e-01, 6.61799131e-05, 3.21385897e-07,
        2.76638639e-06, 1.09542668e-07, 5.34889644e-08, 3.17681454e-08],
       [1.16670456e-07, 9.99926090e-01, 9.62927516e-06, 5.27830025e-06,
        5.61365159e-05, 7.90153763e-07, 1.58850685e-06, 3.10023012e-07],
       [3.61616117e-07, 9.99834895e-01, 6.29741362e-06, 1.04563762e-04,
        4.27038613e-05, 6.99651582e-06, 3.54768781e-06, 5.70213160e-07],
       [2.11416932e-07, 9.99936819e-01, 4.89357953e-05, 5.00438273e-06,
        6.97136556e-06, 8.32286958e-07, 7.61004060e-07, 4.32804342e-07],
       [3.26579539e-07, 9.99762356e-01, 2.97843053e-05, 3.19751825e-06,
        1.97773465e-04, 1.61902858e-06, 3.41090572e-06, 1.41561952e-06],
       [1.45291139e-08, 9.99973416e-01, 2.54983788e-05, 2.62122555e-08,
        5.17174783e-07, 1.84970645e-07, 2.00887822e-08, 1.88774337e-07],
       [7.26437577e-07, 9.99841094e-01, 2.67299038e-05, 7.57465314e-05,
        4.31658227e-05, 5.73444368e-06, 5.17175295e-06, 1.

In [40]:
pred_tags[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [41]:
test_tags[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [42]:
print(classification_report(test_tags, pred_tags))

C:\ProgramData\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\ProgramData\anaconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.61      0.58      0.60       683
         _DT       0.89      0.88      0.89       357
         _LC       0.76      0.59      0.66       324
         _OG       0.65      0.58      0.62       464
         _PS       0.78      0.45      0.57       388
         _TI       0.93      0.72      0.81        76

   micro avg       0.72      0.61      0.66      2292
   macro avg       0.77      0.64      0.69      2292
weighted avg       0.72      0.61      0.66      2292



In [43]:
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

F1-score: 66.2%


In [52]:
# 샐운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = "삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.".split()
new_x = []
for w in new_sentences:
    try:
        new_x.append(word_to_index.get(w, 1))
        
    except KeyError:
        # 모르는 단어일 경우 OOV
        new_x.append(word_to_index["OOV"])

In [53]:
print("새로운 유형의 시퀀스 :", new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding = "post", maxlen = max_len)

새로운 유형의 시퀀스 : [531, 307, 1476, 286, 1507, 6766, 1]


In [54]:
# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis = -1) # 예측된 NER 인덱스 값을 추출
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)

for w, pred in zip(new_sentence, p[0]):
    print("{:10} {:5}".format(w, index_to_ner[pred]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
단어         예측된 NER
--------------------------------------------------
삼성전자       B_OG 
출시         O    
스마트폰       O    
오늘         B_DT 
애플         B_OG 
도전장        I    
내밀다.       I    
